
## 1. Introduction

Base models like Llama 2 can **predict the next token** in a sequence. However, this does not make them particularly useful assistants since they don't reply to instructions. This is why we employ instruction tuning to align their answers with what humans expect. There are two main fine-tuning techniques:

* **Supervised Fine-Tuning** (SFT): Models are trained on a dataset of instructions and responses. It adjusts the weights in the LLM to minimize the difference between the generated answers and ground-truth responses, acting as labels.

* **Reinforcement Learning from Human Feedback** (RLHF): Models learn by interacting with their environment and receiving feedback. They are trained to maximize a reward signal (using [PPO](https://arxiv.org/abs/1707.06347)), which is often derived from human evaluations of model outputs.


In [1]:
# Install the necessary Python packages using pip. The `-q` flag is for quiet output (less verbose).
# `-U` flag is for upgrading all specified packages to the latest version.
!pip install -q -U transformers datasets accelerate peft trl bitsandbytes wandb

# Importing essential libraries for our project
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
from accelerate import Accelerator
import wandb

# Explanation of imported libraries:
# `transformers` - Provides access to pre-trained models and utilities for natural language processing tasks.
# `datasets` - Facilitates easy loading and manipulation of datasets.
# `accelerate` - Simplifies running machine learning models on multi-GPU/multi-TPU setups.
# `wandb` - Weights & Biases; a tool for tracking and visualizing machine learning experiments.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.1/244.1 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━

In [2]:
# Importing `userdata` module from the `google.colab` package
from google.colab import userdata

# Retrieving the 'huggingface' token from the user's secret environment variables defined in the Colab secrets tab.
# This token is typically used for authentication with Hugging Face services, allowing private access or API requests.
hf_token = userdata.get('huggingface')


In [6]:
# Importing the `os` module to interact with the operating system, e.g., accessing environment variables and file paths.
import os

# Importing `torch` which is the PyTorch library, fundamental for tensor operations and neural network training.
import torch

# Importing `load_dataset` function from the `datasets` library to load datasets for training and evaluation.
from datasets import load_dataset

# Importing several classes and functions from the `transformers` library:
from transformers import (
    AutoModelForCausalLM,  # Class to automatically retrieve a pre-trained causal language model appropriate for the given model checkpoint.
    AutoTokenizer,         # Class to automatically retrieve the tokenizer that corresponds to a pre-trained model.
    BitsAndBytesConfig,    # Configuration class for training with `bitsandbytes` for efficient 8-bit optimization.
    TrainingArguments,     # Class to define training parameters such as batch size, learning rate, etc.
    pipeline               # Function to create a pipeline for inference tasks using pre-trained models.
)

# Importing components from the `peft` library for model optimization and fine-tuning:
from peft import (
    LoraConfig,               # Configuration class for Low-Rank Adaptation parameters.
    PeftModel,                # Class representing a model prepared for Parameter Efficient Fine-Tuning.
    prepare_model_for_kbit_training  # Function to prepare a model specifically for efficient training using reduced precision.
)

# Importing `SFTTrainer` from the `trl` library to implement Supervised Fine-Tuning using Transformer models.
from trl import SFTTrainer


In [7]:
# Base model identifier on Hugging Face Model Hub. This is a pre-trained model that we will fine-tune.
base_model = 'NousResearch/Llama-2-7b-hf'

# This is the identifier for the model that will be saved after fine-tuning. It's named according to the dataset used.
new_model = 'llama-2-7b-platypus'

# Loading a dataset that has been previously uploaded to the Hugging Face Hub by the user 'anishgillella'.
# The dataset is specifically split to use only the training portion for the fine-tuning process.
dataset = load_dataset('anishgillella/mini-platypus', split='train')

# Loading the tokenizer associated with the base model. Tokenizers are used to convert text to a format that the model can understand.
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)

# Setting the padding token to be the same as the end-of-sentence (EOS) token.
# This is often done in models that use a single token for both purposes, helping the model understand when a sentence ends.
tokenizer.pad_token = tokenizer.eos_token

# Correcting a typo in the property name for setting the padding strategy.
# It should be `padding_side` rather than `paddin_side`. This sets the padding direction when processing input sequences.
tokenizer.padding_side = "right"  # Ensures that padding is added to the right of the sequences.

# Explanation:
# Padding each token sequence to the same length is crucial for batch processing in neural networks,
# as it allows for efficient computation by ensuring all input tensors are of uniform shape.


Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [9]:
# Configuring Quantization with BitsAndBytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,               # Enables loading model weights in 4-bit precision, reducing memory usage.
    bnb_4bit_quant_type="nf4",       # Specifies the type of 4-bit quantization; "nf4" stands for "Noise-Free 4-bit".
    bnb_4bit_compute_type=torch.float16,  # Sets the compute type to 16-bit floating point, balancing precision and speed.
    bnb_4bit_double_quant=True       # Enables double quantization, potentially enhancing accuracy at the cost of speed.
)

# Configuration for LoRA (Low-Rank Adaptation), a method to adapt pre-trained models with minimal extra parameters
peft_config = LoraConfig(
    lora_alpha=32,                   # The expansion factor for LoRA layers, determining the size of the trainable parameters.
    lora_dropout=0.05,               # Dropout rate in LoRA layers to prevent overfitting.
    r=16,                            # Rank of the adaptation matrix, a key hyperparameter in LoRA to control learning capacity.
    bias="none",                     # Whether to include bias in LoRA layers. Here, it's turned off.
    task_type="CAUSAL_LM",           # Specifies that the model adaptation is for a causal language model.
    target_modules=["up_proj", "down_proj", "k_proj", "q_proj", "v_proj", "o_proj"]  # Specifies the transformer modules to adapt.
    # More modules adapted increases flexibility and potential capacity of the model but requires more parameters.
)

# Loading the base causal language model from Hugging Face with the specified BitsAndBytes quantization configuration.
# It also specifies device mapping for model parts, potentially used for distributing the model across different hardware.
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": 0}  # Maps the model or its parts to a specific device, here device '0' (typically GPU).
)

# Prepare the model for training using k-bit quantization, a technique for reducing model size and computational needs.
model = prepare_model_for_kbit_training(model)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [23]:
dataset

Dataset({
    features: ['train'],
    num_rows: 1000
})

In [25]:
print(dataset['train'].head())

{'train': {'instruction': Value(dtype='string', id=None), 'output': Value(dtype='string', id=None)}}


In [34]:
dataset.head()

train
0  {'instruction': '### Instruction:
Let's come u...
1  {'instruction': '### Instruction:
Let
$$p(x,y)...
2  {'instruction': '### Instruction:
Given the co...
3  {'instruction': '### Instruction:
Find the are...
4  {'instruction': '### Instruction:
Let $P$ be t...

In [ ]:
import pandas as pd

# TrainingArguments configures the parameters for training the model using Hugging Face's Transformers library
training_arguments = TrainingArguments(
    output_dir="./results",          # Directory where the output files will be saved.
    num_train_epochs=1,             # Number of epochs to train for.
    per_device_train_batch_size=10, # Batch size per device during training.
    gradient_accumulation_steps=1,  # Number of steps to accumulate gradients before performing a backward/update pass.
    evaluation_strategy="steps",    # Evaluation is done (and model saved) every `eval_steps`.
    eval_steps=1000,                # Number of steps before performing evaluation.
    logging_steps=1,                # Frequency of logging predictions.
    optim="paged_adamw_8bit",       # Optimizer used for training, designed for efficient memory usage.
    learning_rate=2e-4,             # Learning rate for the optimizer.
    lr_scheduler_type="linear",     # Type of learning rate scheduler.
    warmup_steps=10,                # Number of steps to perform learning rate warmup.
    report_to="wandb",              # Enables logging to Weights & Biases for monitoring.
    max_steps=2,                    # Maximum number of training steps to perform. Remove this for full training.
)

# Supervised Fine-Tuning Trainer (SFTTrainer) for causal language models
trainer = SFTTrainer(
    model=model,                         # Pre-trained language model being fine-tuned.
    train_dataset=train_dataset,          # Training dataset.
    eval_dataset=eval_dataset,            # Evaluation dataset.
    peft_config=peft_config,              # Configuration for PEFT (Parametric Efficient Fine-Tuning).
    dataset_text_field='instruction',     # Field in the dataset containing text to model.
    max_seq_length=512,                   # Maximum sequence length the model will handle.
    tokenizer=tokenizer,                  # Tokenizer used for encoding the texts.
    args=training_arguments,              # Training arguments defined above.
)

# Execute the training process
trainer.train()

# Save the fine-tuned model to the specified directory
trainer.model.save_pretrained(new_model)
